In [1]:
#!/usr/bin/env python
# coding: utf-8
# run using: sbatch --array=0-9 7.9-get-predictions-from-BERT.sh

import sys
import os

# column = sys.argv[1]
# column = 'is_unemployed'


####################################################################################################################################
# loading the model
####################################################################################################################################


import time

start_time = time.time()
from transformers import BertTokenizer
from pathlib import Path
import torch

from box import Box
import pandas as pd
import collections

from tqdm import tqdm, trange
# import sys
import random
import numpy as np
# import apex
from sklearn.model_selection import train_test_split

import datetime

import sys
import pickle
import os

import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from ast import literal_eval
from sklearn.metrics import classification_report
import gensim.downloader as api
from sklearn.preprocessing import scale


# sys.path.append('../')

# from fast_bert.modeling import BertForMultiLabelSequenceClassification
# from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, \
#     convert_examples_to_features
# from fast_bert.learner_cls import BertLearner
# # from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc, accuracy
# from fast_bert.metrics import *
import matplotlib.pyplot as plt

# torch.cuda.empty_cache()

pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

root_path='/scratch/da2734/twitter/jobs/running_on_200Msamples/'




# def create_model(columnm, epoch):
#     if not os.path.exists('/scratch/da2734/twitter/jobs/running_on_200Msamples/logs/log_binary_pos_neg_{}/'.format(column)):
#         os.makedirs('/scratch/da2734/twitter/jobs/running_on_200Msamples/logs/log_binary_pos_neg_{}/'.format(column))

#     LOG_PATH = Path('/scratch/da2734/twitter/jobs/running_on_200Msamples/logs/log_binary_pos_neg_{}/'.format(column))
#     print('LOG_PATH', LOG_PATH)
#     DATA_PATH = Path('/scratch/da2734/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/')
#     LABEL_PATH = Path('/scratch/da2734/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/')
#     OUTPUT_PATH = Path('/scratch/da2734/twitter/jobs/training_binary/models_may20_9Klabels/output_{}'.format(column))
#     FINETUNED_PATH = None

#     args = Box({
#         "run_text": "100Msamples",
#         "train_size": -1,
#         "val_size": -1,
#         "log_path": LOG_PATH,
#         "full_data_dir": DATA_PATH,
#         "data_dir": DATA_PATH,
#         "task_name": "labor_market_classification",
#         "no_cuda": False,
#         #     "bert_model": BERT_PRETRAINED_PATH,
#         "output_dir": OUTPUT_PATH,
#         "max_seq_length": 512,
#         "do_train": True,
#         "do_eval": True,
#         "do_lower_case": True,
#         "train_batch_size": 8,
#         "eval_batch_size": 16,
#         "learning_rate": 5e-5,
#         "num_train_epochs": 100,
#         "warmup_proportion": 0.0,
#         "no_cuda": False,
#         "local_rank": -1,
#         "seed": 42,
#         "gradient_accumulation_steps": 1,
#         "optimize_on_cpu": False,
#         "fp16": False,
#         "fp16_opt_level": "O1",
#         "weight_decay": 0.0,
#         "adam_epsilon": 1e-8,
#         "max_grad_norm": 1.0,
#         "max_steps": -1,
#         "warmup_steps": 500,
#         "logging_steps": 50,
#         "eval_all_checkpoints": True,
#         "overwrite_output_dir": True,
#         "overwrite_cache": True,
#         "seed": 42,
#         "loss_scale": 128,
#         "task_name": 'intent',
#         "model_name": 'bert-base-uncased',
#         "model_type": 'bert'
#     })

#     import logging

#     logfile = str(LOG_PATH / 'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

#     logging.basicConfig(
#         level=logging.INFO,
#         format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
#         datefmt='%m/%d/%Y %H:%M:%S',
#         handlers=[
#             logging.FileHandler(logfile),
#             logging.StreamHandler(sys.stdout)
#         ])

#     logger = logging.getLogger()

#     logger.info(args)

#     device = torch.device('cuda')
#     if torch.cuda.device_count() > 1:
#         args.multi_gpu = True
#     else:
#         args.multi_gpu = False

#     label_cols = ['class']

#     databunch = BertDataBunch(
#         args['data_dir'],
#         LABEL_PATH,
#         args.model_name,
#         train_file='train_{}.csv'.format(column),
#         val_file='val_{}.csv'.format(column),
#         label_file='label_{}.csv'.format(column),
#         # test_data='test.csv',
#         text_col="text",  # this is the name of the column in the train file that containts the tweet text
#         label_col=label_cols,
#         batch_size_per_gpu=args['train_batch_size'],
#         max_seq_length=args['max_seq_length'],
#         multi_gpu=args.multi_gpu,
#         multi_label=False,
#         model_type=args.model_type)

#     num_labels = len(databunch.labels)
#     print('num_labels', num_labels)

#     print('time taken to load all this stuff:', str(time.time() - start_time), 'seconds')

#     # metrics defined: https://github.com/kaushaltrivedi/fast-bert/blob/d89e2aa01d948d6d3cdea7ad106bf5792fea7dfa/fast_bert/metrics.py
#     metrics = []
#     # metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
#     # metrics.append({'name': 'roc_auc', 'function': roc_auc})
#     # metrics.append({'name': 'fbeta', 'function': fbeta})
#     metrics.append({'name': 'accuracy', 'function': accuracy})
#     metrics.append({'name': 'roc_auc_save_to_plot_binary', 'function': roc_auc_save_to_plot_binary})
#     # metrics.append({'name': 'accuracy_multilabel', 'function': accuracy_multilabel})

#     learner = BertLearner.from_pretrained_model(
#         databunch,
#         pretrained_path='/scratch/da2734/twitter/jobs/training_binary/models_may20_9Klabels/output_{}/model_out_{}/'.format(column, epoch),
#         metrics=metrics,
#         device=device,
#         logger=logger,
#         output_dir=args.output_dir,
#         finetuned_wgts_path=FINETUNED_PATH,
#         warmup_steps=args.warmup_steps,
#         multi_gpu=args.multi_gpu,
#         is_fp16=args.fp16,
#         multi_label=False,
#         logging_steps=0)

#     return learner


# best_epochs = {
#     'is_hired_1mo':8,
#     'lost_job_1mo':5,
#     'job_offer':4,
#     'is_unemployed':3,
#     'job_search':6
# }

# best_epochs = {
#     'is_hired_1mo':6,
#     'lost_job_1mo':4,
#     'job_offer':3,
#     'is_unemployed':3,
#     'job_search':4
# }


def get_env_var(varname, default):
    if os.environ.get(varname) != None:
        var = int(os.environ.get(varname))
        print(varname, ':', var)
    else:
        var = default
        print(varname, ':', var, '(Default)')
    return var


# Choose Number of Nodes To Distribute Credentials: e.g. jobarray=0-4, cpu_per_task=20, credentials = 90 (<100)
# SLURM_JOB_ID = get_env_var('SLURM_JOB_ID', 0)
# SLURM_ARRAY_TASK_ID = get_env_var('SLURM_ARRAY_TASK_ID', 0)
# SLURM_ARRAY_TASK_COUNT = get_env_var('SLURM_ARRAY_TASK_COUNT', 1)

SLURM_JOB_ID = 123123123
SLURM_ARRAY_TASK_ID = 10
SLURM_ARRAY_TASK_COUNT = 500


print('SLURM_JOB_ID', SLURM_JOB_ID)
print('SLURM_ARRAY_TASK_ID', SLURM_ARRAY_TASK_ID)
print('SLURM_ARRAY_TASK_COUNT', SLURM_ARRAY_TASK_COUNT)


# ####################################################################################################################################
# # loading data
# ####################################################################################################################################

import time
import pyarrow.parquet as pq
from glob import glob
import os
import numpy as np

path_to_data='/scratch/spf248/twitter/data/classification/US/'


# print('Load Filtered Tweets:')
# # filtered contains 8G of data!!
# start_time = time.time()

# paths_to_filtered=list(np.array_split(
#                         # glob(os.path.join(path_to_data,'filtered','*.parquet')),
#                         # glob(os.path.join(path_to_data,'filtered_10perct_sample','*.parquet')),
#                         glob(os.path.join(path_to_data,'filtered_1perct_sample','*.parquet')),
#                         SLURM_ARRAY_TASK_COUNT)[SLURM_ARRAY_TASK_ID]
#                        )
# print('#files:', len(paths_to_filtered))

# tweets_filtered=pd.DataFrame()
# for file in paths_to_filtered:
#     print(file)
#     tweets_filtered=pd.concat([tweets_filtered,pd.read_parquet(file)[['tweet_id','text']]])
#     print(tweets_filtered.shape)

#     # break

# # tweets_filtered = tweets_filtered[:100]

# print('time taken to load keyword filtered sample:', str(time.time() - start_time), 'seconds')
# print(tweets_filtered.shape)


print('Load Random Tweets:')
# random contains 7.3G of data!!
start_time = time.time()

paths_to_random=list(np.array_split(
                        # glob(os.path.join(path_to_data,'random','*.parquet')),
                        # glob(os.path.join(path_to_data,'random_10perct_sample','*.parquet')),
                        glob(os.path.join(path_to_data,'random_1perct_sample','*.parquet')),
                        SLURM_ARRAY_TASK_COUNT)[SLURM_ARRAY_TASK_ID])
print('#files:', len(paths_to_random))

tweets_random=pd.DataFrame()
for file in paths_to_random:
    print(file)
    tweets_random=pd.concat([tweets_random,pd.read_parquet(file)[['tweet_id','text']]])
    print(tweets_random.shape)

    break

tweets_random = tweets_random[:100]

print('time taken to load random sample:', str(time.time() - start_time), 'seconds')
print(tweets_random.shape)


def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

def get_w2v_general(tweet, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count
        return vec
    elif aggregation == 'sum':
        return vec

import os
print('GENSIM_DATA_DIR', os.environ['GENSIM_DATA_DIR'] )
    
start_time = time.time()
print('loading glove')
glove_twitter = api.load("glove-twitter-200")
print('time taken:', str(time.time() - start_time), 'seconds')    

start_time = time.time()
print('calculating embeddings')    
random_data_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in tweets_random["text"]]))
print('time taken:', str(time.time() - start_time), 'seconds')
print('per tweet:', (time.time() - start_time)/tweets_random.shape[0], 'seconds')


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:

    print('\n\n!!!!!', column)

#     start = time.time()
#     learner = create_model(column, best_epochs[column])
#     print('load model:', str(time.time() - start_time), 'seconds')

#     print('Predictions of Filtered Tweets:')
#     start_time = time.time()
#     predictions_filtered = learner.predict_batch(tweets_filtered['text'].values.tolist())
#     print('time taken:', str(time.time() - start_time), 'seconds')
#     print('per tweet:', (time.time() - start_time)/tweets_filtered.shape[0], 'seconds')

#     # In[ ]:

    data_path = "/scratch/da2734/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/"
    print("************ {} ************".format(column))

    train_file_name = "train_{}.csv".format(column)
    val_file_name = "val_{}.csv".format(column)
    #download data
    df_train = pd.read_csv(os.path.join(data_path, train_file_name))
    print(df_train.head())
    df_val = pd.read_csv(os.path.join(data_path, val_file_name))
    #create embeddings
    train_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in df_train["text"]]))
    validation_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in df_val["text"]]))
    #train
    clf = LogisticRegression(max_iter=1000)
    clf.fit(train_vecs_glove_mean,df_train["class"])
    #evaluate
    df_val["class_predict"] = clf.predict(validation_vecs_glove_mean)
    TP, FP, TN, FN = perf_measure(df_val["class"], df_val["class_predict"])
    print("Precision: ", TP/(TP+FP))
    print("Recall: ", TP/(TP+FN))





    print('Predictions of Random Tweets:')
    start_time = time.time()
    #     predictions_random = learner.predict_batch(tweets_random['text'].values.tolist())
    predictions_random = clf.predict_proba(random_data_vecs_glove_mean)

    print(type(predictions_random))
    # print(predictions_random)

    print('time taken:', str(time.time() - start_time), 'seconds')
    print('per tweet:', (time.time() - start_time)/tweets_random.shape[0], 'seconds')

    # In[ ]:


    #     print('Save Predictions of Filtered Tweets:')
    #     start_time = time.time()



    #     df_filtered = predictions_filtered.set_index(tweets_filtered.tweet_id).rename(columns={
    #             '0':'pos_model',
    #             '1':'neg_model',
    #     })

    if not os.path.exists(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column)):
        print('>>>> directory doesnt exists')
        os.makedirs(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column))

    #     # if not os.path.exists(os.path.join(root_path,'pred_output_10pct_sample', column)):
    #     #     os.makedirs(os.path.join(root_path,'pred_output_10pct_sample', column))

    #     # if not os.path.exists(os.path.join(root_path,'pred_output_full', column)):
    #     #     os.makedirs(os.path.join(root_path,'pred_output_full', column))

    #     df_filtered.to_csv(
    #             # os.path.join(root_path,'pred_output', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
    #             os.path.join(root_path,'pred_output_1pct_sample', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
    #         )

    #     print(os.path.join(root_path,'pred_output_1pct_sample', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv'), 'saved')

    #     print('time taken:', str(time.time() - start_time), 'seconds')






    print('Save Predictions of Random Tweets:')
    start_time = time.time()
    # predictions_random_df = pd.DataFrame(data=predictions_random, columns = ['neg', 'pos'])
    predictions_random_df = pd.DataFrame(data=predictions_random, columns = ['neg_model', 'pos_model'])
    df_random = predictions_random_df.set_index(tweets_random.tweet_id)
    # df_random = predictions_random.set_index(tweets_random.tweet_id).rename(columns={
    #         '0':'pos_model',
    #         '1':'neg_model',
    # })

    # if not os.path.exists(os.path.join(root_path,'pred_output_10pct_sample', column)):
    #     os.makedirs(os.path.join(root_path,'pred_output_10pct_sample', column))

    df_random.to_csv(
        # os.path.join(root_path,'pred_output', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
        os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
        )

    print(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv'), 'saved')

    print('time taken:', str(time.time() - start_time), 'seconds')


    break

    

/scratch/da2734/pyenv_dval_wb_twitter/py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:63: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


SLURM_JOB_ID 123123123
SLURM_ARRAY_TASK_ID 10
SLURM_ARRAY_TASK_COUNT 500
Load Random Tweets:
#files: 1
/scratch/spf248/twitter/data/classification/US/random_1perct_sample/part-00974-1c1e6466-49fa-411b-beb0-276d14cdffab-c000.snappy.parquet
(42521, 2)
time taken to load random sample: 0.05587458610534668 seconds
(100, 2)
GENSIM_DATA_DIR /scratch/da2734/twitter/code/11-baseline/logit_glove/downloaded
loading glove
time taken: 222.1632137298584 seconds
calculating embeddings
time taken: 0.00710606575012207 seconds
per tweet: 7.481813430786133e-05 seconds


!!!!! is_unemployed
************ is_unemployed ************
     id  \
0  2772   
1  1205   
2  4512   
3  7249   
4  9453   

                                                                                                                                        text  \
0  I was late again got work today. They gonna fire the shit outta me one day                                                                  
1  Got laid off today :/#w

In [1]:
#!/usr/bin/env python
# coding: utf-8
# run using: sbatch --array=0-9 7.9-get-predictions-from-BERT.sh

import sys
import os

# column = sys.argv[1]
# column = 'is_unemployed'


####################################################################################################################################
# loading the model
####################################################################################################################################


import time

start_time = time.time()
from transformers import BertTokenizer
from pathlib import Path
import torch

from box import Box
import pandas as pd
import collections

from tqdm import tqdm, trange
# import sys
import random
import numpy as np
# import apex
from sklearn.model_selection import train_test_split

import datetime

import sys
import pickle
import os

import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from ast import literal_eval
from sklearn.metrics import classification_report
import gensim.downloader as api
from sklearn.preprocessing import scale


# sys.path.append('../')

# from fast_bert.modeling import BertForMultiLabelSequenceClassification
# from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, \
#     convert_examples_to_features
# from fast_bert.learner_cls import BertLearner
# # from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc, accuracy
# from fast_bert.metrics import *
import matplotlib.pyplot as plt

# torch.cuda.empty_cache()

pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

root_path='/scratch/da2734/twitter/jobs/running_on_200Msamples/'




# def create_model(columnm, epoch):
#     if not os.path.exists('/scratch/da2734/twitter/jobs/running_on_200Msamples/logs/log_binary_pos_neg_{}/'.format(column)):
#         os.makedirs('/scratch/da2734/twitter/jobs/running_on_200Msamples/logs/log_binary_pos_neg_{}/'.format(column))

#     LOG_PATH = Path('/scratch/da2734/twitter/jobs/running_on_200Msamples/logs/log_binary_pos_neg_{}/'.format(column))
#     print('LOG_PATH', LOG_PATH)
#     DATA_PATH = Path('/scratch/da2734/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/')
#     LABEL_PATH = Path('/scratch/da2734/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/')
#     OUTPUT_PATH = Path('/scratch/da2734/twitter/jobs/training_binary/models_may20_9Klabels/output_{}'.format(column))
#     FINETUNED_PATH = None

#     args = Box({
#         "run_text": "100Msamples",
#         "train_size": -1,
#         "val_size": -1,
#         "log_path": LOG_PATH,
#         "full_data_dir": DATA_PATH,
#         "data_dir": DATA_PATH,
#         "task_name": "labor_market_classification",
#         "no_cuda": False,
#         #     "bert_model": BERT_PRETRAINED_PATH,
#         "output_dir": OUTPUT_PATH,
#         "max_seq_length": 512,
#         "do_train": True,
#         "do_eval": True,
#         "do_lower_case": True,
#         "train_batch_size": 8,
#         "eval_batch_size": 16,
#         "learning_rate": 5e-5,
#         "num_train_epochs": 100,
#         "warmup_proportion": 0.0,
#         "no_cuda": False,
#         "local_rank": -1,
#         "seed": 42,
#         "gradient_accumulation_steps": 1,
#         "optimize_on_cpu": False,
#         "fp16": False,
#         "fp16_opt_level": "O1",
#         "weight_decay": 0.0,
#         "adam_epsilon": 1e-8,
#         "max_grad_norm": 1.0,
#         "max_steps": -1,
#         "warmup_steps": 500,
#         "logging_steps": 50,
#         "eval_all_checkpoints": True,
#         "overwrite_output_dir": True,
#         "overwrite_cache": True,
#         "seed": 42,
#         "loss_scale": 128,
#         "task_name": 'intent',
#         "model_name": 'bert-base-uncased',
#         "model_type": 'bert'
#     })

#     import logging

#     logfile = str(LOG_PATH / 'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

#     logging.basicConfig(
#         level=logging.INFO,
#         format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
#         datefmt='%m/%d/%Y %H:%M:%S',
#         handlers=[
#             logging.FileHandler(logfile),
#             logging.StreamHandler(sys.stdout)
#         ])

#     logger = logging.getLogger()

#     logger.info(args)

#     device = torch.device('cuda')
#     if torch.cuda.device_count() > 1:
#         args.multi_gpu = True
#     else:
#         args.multi_gpu = False

#     label_cols = ['class']

#     databunch = BertDataBunch(
#         args['data_dir'],
#         LABEL_PATH,
#         args.model_name,
#         train_file='train_{}.csv'.format(column),
#         val_file='val_{}.csv'.format(column),
#         label_file='label_{}.csv'.format(column),
#         # test_data='test.csv',
#         text_col="text",  # this is the name of the column in the train file that containts the tweet text
#         label_col=label_cols,
#         batch_size_per_gpu=args['train_batch_size'],
#         max_seq_length=args['max_seq_length'],
#         multi_gpu=args.multi_gpu,
#         multi_label=False,
#         model_type=args.model_type)

#     num_labels = len(databunch.labels)
#     print('num_labels', num_labels)

#     print('time taken to load all this stuff:', str(time.time() - start_time), 'seconds')

#     # metrics defined: https://github.com/kaushaltrivedi/fast-bert/blob/d89e2aa01d948d6d3cdea7ad106bf5792fea7dfa/fast_bert/metrics.py
#     metrics = []
#     # metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
#     # metrics.append({'name': 'roc_auc', 'function': roc_auc})
#     # metrics.append({'name': 'fbeta', 'function': fbeta})
#     metrics.append({'name': 'accuracy', 'function': accuracy})
#     metrics.append({'name': 'roc_auc_save_to_plot_binary', 'function': roc_auc_save_to_plot_binary})
#     # metrics.append({'name': 'accuracy_multilabel', 'function': accuracy_multilabel})

#     learner = BertLearner.from_pretrained_model(
#         databunch,
#         pretrained_path='/scratch/da2734/twitter/jobs/training_binary/models_may20_9Klabels/output_{}/model_out_{}/'.format(column, epoch),
#         metrics=metrics,
#         device=device,
#         logger=logger,
#         output_dir=args.output_dir,
#         finetuned_wgts_path=FINETUNED_PATH,
#         warmup_steps=args.warmup_steps,
#         multi_gpu=args.multi_gpu,
#         is_fp16=args.fp16,
#         multi_label=False,
#         logging_steps=0)

#     return learner


# best_epochs = {
#     'is_hired_1mo':8,
#     'lost_job_1mo':5,
#     'job_offer':4,
#     'is_unemployed':3,
#     'job_search':6
# }

# best_epochs = {
#     'is_hired_1mo':6,
#     'lost_job_1mo':4,
#     'job_offer':3,
#     'is_unemployed':3,
#     'job_search':4
# }


def get_env_var(varname, default):
    if os.environ.get(varname) != None:
        var = int(os.environ.get(varname))
        print(varname, ':', var)
    else:
        var = default
        print(varname, ':', var, '(Default)')
    return var


# Choose Number of Nodes To Distribute Credentials: e.g. jobarray=0-4, cpu_per_task=20, credentials = 90 (<100)
# SLURM_JOB_ID = get_env_var('SLURM_JOB_ID', 0)
# SLURM_ARRAY_TASK_ID = get_env_var('SLURM_ARRAY_TASK_ID', 0)
# SLURM_ARRAY_TASK_COUNT = get_env_var('SLURM_ARRAY_TASK_COUNT', 1)

SLURM_JOB_ID = 123123123
SLURM_ARRAY_TASK_ID = 10
SLURM_ARRAY_TASK_COUNT = 500


print('SLURM_JOB_ID', SLURM_JOB_ID)
print('SLURM_ARRAY_TASK_ID', SLURM_ARRAY_TASK_ID)
print('SLURM_ARRAY_TASK_COUNT', SLURM_ARRAY_TASK_COUNT)


# ####################################################################################################################################
# # loading data
# ####################################################################################################################################

import time
import pyarrow.parquet as pq
from glob import glob
import os
import numpy as np

path_to_data='/scratch/spf248/twitter/data/classification/US/'


# print('Load Filtered Tweets:')
# # filtered contains 8G of data!!
# start_time = time.time()

# paths_to_filtered=list(np.array_split(
#                         # glob(os.path.join(path_to_data,'filtered','*.parquet')),
#                         # glob(os.path.join(path_to_data,'filtered_10perct_sample','*.parquet')),
#                         glob(os.path.join(path_to_data,'filtered_1perct_sample','*.parquet')),
#                         SLURM_ARRAY_TASK_COUNT)[SLURM_ARRAY_TASK_ID]
#                        )
# print('#files:', len(paths_to_filtered))

# tweets_filtered=pd.DataFrame()
# for file in paths_to_filtered:
#     print(file)
#     tweets_filtered=pd.concat([tweets_filtered,pd.read_parquet(file)[['tweet_id','text']]])
#     print(tweets_filtered.shape)

#     # break

# # tweets_filtered = tweets_filtered[:100]

# print('time taken to load keyword filtered sample:', str(time.time() - start_time), 'seconds')
# print(tweets_filtered.shape)


print('Load Random Tweets:')
# random contains 7.3G of data!!
start_time = time.time()

paths_to_random=list(np.array_split(
                        # glob(os.path.join(path_to_data,'random','*.parquet')),
                        # glob(os.path.join(path_to_data,'random_10perct_sample','*.parquet')),
                        glob(os.path.join(path_to_data,'random_1perct_sample','*.parquet')),
                        SLURM_ARRAY_TASK_COUNT)[SLURM_ARRAY_TASK_ID])
print('#files:', len(paths_to_random))

tweets_random=pd.DataFrame()
for file in paths_to_random:
    print(file)
    tweets_random=pd.concat([tweets_random,pd.read_parquet(file)[['tweet_id','text']]])
    print(tweets_random.shape)

    break

tweets_random = tweets_random[:100]

print('time taken to load random sample:', str(time.time() - start_time), 'seconds')
print(tweets_random.shape)


def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

def get_w2v_general(tweet, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count
        return vec
    elif aggregation == 'sum':
        return vec

import os
print('GENSIM_DATA_DIR', os.environ['GENSIM_DATA_DIR'] )
    
start_time = time.time()
print('loading glove')
glove_twitter = api.load("glove-twitter-200")
print('time taken:', str(time.time() - start_time), 'seconds')    

start_time = time.time()
print('calculating embeddings')    
random_data_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in tweets_random["text"]]))
print('time taken:', str(time.time() - start_time), 'seconds')
print('per tweet:', (time.time() - start_time)/tweets_random.shape[0], 'seconds')


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:

    print('\n\n!!!!!', column)

#     start = time.time()
#     learner = create_model(column, best_epochs[column])
#     print('load model:', str(time.time() - start_time), 'seconds')

#     print('Predictions of Filtered Tweets:')
#     start_time = time.time()
#     predictions_filtered = learner.predict_batch(tweets_filtered['text'].values.tolist())
#     print('time taken:', str(time.time() - start_time), 'seconds')
#     print('per tweet:', (time.time() - start_time)/tweets_filtered.shape[0], 'seconds')

#     # In[ ]:

    data_path = "/scratch/da2734/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/"
    print("************ {} ************".format(column))

    train_file_name = "train_{}.csv".format(column)
    val_file_name = "val_{}.csv".format(column)
    #download data
    df_train = pd.read_csv(os.path.join(data_path, train_file_name))
    print(df_train.head())
    df_val = pd.read_csv(os.path.join(data_path, val_file_name))
    #create embeddings
    train_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in df_train["text"]]))
    validation_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in df_val["text"]]))
    #train
    clf = LogisticRegression(max_iter=1000)
    clf.fit(train_vecs_glove_mean,df_train["class"])
    #evaluate
    df_val["class_predict"] = clf.predict(validation_vecs_glove_mean)
    TP, FP, TN, FN = perf_measure(df_val["class"], df_val["class_predict"])
    print("Precision: ", TP/(TP+FP))
    print("Recall: ", TP/(TP+FN))





    print('Predictions of Random Tweets:')
    start_time = time.time()
    #     predictions_random = learner.predict_batch(tweets_random['text'].values.tolist())
    predictions_random = clf.predict_proba(random_data_vecs_glove_mean)

    print(type(predictions_random))
    # print(predictions_random)

    print('time taken:', str(time.time() - start_time), 'seconds')
    print('per tweet:', (time.time() - start_time)/tweets_random.shape[0], 'seconds')

    # In[ ]:


    #     print('Save Predictions of Filtered Tweets:')
    #     start_time = time.time()



    #     df_filtered = predictions_filtered.set_index(tweets_filtered.tweet_id).rename(columns={
    #             '0':'pos_model',
    #             '1':'neg_model',
    #     })

    if not os.path.exists(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column)):
        print('>>>> directory doesnt exists')
        os.makedirs(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column))

    #     # if not os.path.exists(os.path.join(root_path,'pred_output_10pct_sample', column)):
    #     #     os.makedirs(os.path.join(root_path,'pred_output_10pct_sample', column))

    #     # if not os.path.exists(os.path.join(root_path,'pred_output_full', column)):
    #     #     os.makedirs(os.path.join(root_path,'pred_output_full', column))

    #     df_filtered.to_csv(
    #             # os.path.join(root_path,'pred_output', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
    #             os.path.join(root_path,'pred_output_1pct_sample', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
    #         )

    #     print(os.path.join(root_path,'pred_output_1pct_sample', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv'), 'saved')

    #     print('time taken:', str(time.time() - start_time), 'seconds')






    print('Save Predictions of Random Tweets:')
    start_time = time.time()
    # predictions_random_df = pd.DataFrame(data=predictions_random, columns = ['neg', 'pos'])
    predictions_random_df = pd.DataFrame(data=predictions_random, columns = ['neg_model', 'pos_model'])
    df_random = predictions_random_df.set_index(tweets_random.tweet_id)
    # df_random = predictions_random.set_index(tweets_random.tweet_id).rename(columns={
    #         '0':'pos_model',
    #         '1':'neg_model',
    # })

    # if not os.path.exists(os.path.join(root_path,'pred_output_10pct_sample', column)):
    #     os.makedirs(os.path.join(root_path,'pred_output_10pct_sample', column))

    df_random.to_csv(
        # os.path.join(root_path,'pred_output', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
        os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
        )

    print(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv'), 'saved')

    print('time taken:', str(time.time() - start_time), 'seconds')


    # break

    

/scratch/da2734/pyenv_dval_wb_twitter/py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:63: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


SLURM_JOB_ID 123123123
SLURM_ARRAY_TASK_ID 10
SLURM_ARRAY_TASK_COUNT 500
Load Random Tweets:
#files: 1
/scratch/spf248/twitter/data/classification/US/random_1perct_sample/part-00974-1c1e6466-49fa-411b-beb0-276d14cdffab-c000.snappy.parquet
(42521, 2)
time taken to load random sample: 0.05513167381286621 seconds
(100, 2)
GENSIM_DATA_DIR /scratch/da2734/twitter/code/11-baseline/logit_glove/downloaded
loading glove
time taken: 220.26266598701477 seconds
calculating embeddings
time taken: 0.0072174072265625 seconds
per tweet: 7.601737976074218e-05 seconds


!!!!! is_unemployed


!!!!! lost_job_1mo


!!!!! job_search


!!!!! is_hired_1mo


!!!!! job_offer


In [3]:
# huh
data_path = "/scratch/da2734/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/"
print("************ {} ************".format(column))

train_file_name = "train_{}.csv".format(column)
val_file_name = "val_{}.csv".format(column)
#download data
df_train = pd.read_csv(os.path.join(data_path, train_file_name))
print(df_train.head())
df_val = pd.read_csv(os.path.join(data_path, val_file_name))
#create embeddings
train_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in df_train["text"]]))
validation_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in df_val["text"]]))
#train
clf = LogisticRegression(max_iter=1000)
clf.fit(train_vecs_glove_mean,df_train["class"])
#evaluate
df_val["class_predict"] = clf.predict(validation_vecs_glove_mean)
TP, FP, TN, FN = perf_measure(df_val["class"], df_val["class_predict"])
print("Precision: ", TP/(TP+FP))
print("Recall: ", TP/(TP+FN))





print('Predictions of Random Tweets:')
start_time = time.time()
#     predictions_random = learner.predict_batch(tweets_random['text'].values.tolist())
predictions_random = clf.predict_proba(random_data_vecs_glove_mean)

print(type(predictions_random))
# print(predictions_random)

print('time taken:', str(time.time() - start_time), 'seconds')
print('per tweet:', (time.time() - start_time)/tweets_random.shape[0], 'seconds')

# In[ ]:


#     print('Save Predictions of Filtered Tweets:')
#     start_time = time.time()



#     df_filtered = predictions_filtered.set_index(tweets_filtered.tweet_id).rename(columns={
#             '0':'pos_model',
#             '1':'neg_model',
#     })

if not os.path.exists(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column)):
    print('>>>> directory doesnt exists')
    os.makedirs(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column))

#     # if not os.path.exists(os.path.join(root_path,'pred_output_10pct_sample', column)):
#     #     os.makedirs(os.path.join(root_path,'pred_output_10pct_sample', column))

#     # if not os.path.exists(os.path.join(root_path,'pred_output_full', column)):
#     #     os.makedirs(os.path.join(root_path,'pred_output_full', column))

#     df_filtered.to_csv(
#             # os.path.join(root_path,'pred_output', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
#             os.path.join(root_path,'pred_output_1pct_sample', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
#         )

#     print(os.path.join(root_path,'pred_output_1pct_sample', column, 'filtered'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv'), 'saved')

#     print('time taken:', str(time.time() - start_time), 'seconds')






print('Save Predictions of Random Tweets:')
start_time = time.time()
# predictions_random_df = pd.DataFrame(data=predictions_random, columns = ['neg', 'pos'])
predictions_random_df = pd.DataFrame(data=predictions_random, columns = ['neg_model', 'pos_model'])
df_random = predictions_random_df.set_index(tweets_random.tweet_id)
# df_random = predictions_random.set_index(tweets_random.tweet_id).rename(columns={
#         '0':'pos_model',
#         '1':'neg_model',
# })

# if not os.path.exists(os.path.join(root_path,'pred_output_10pct_sample', column)):
#     os.makedirs(os.path.join(root_path,'pred_output_10pct_sample', column))

df_random.to_csv(
    # os.path.join(root_path,'pred_output', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
    os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv')
    )

print(os.path.join(root_path,'pred_output_1pct_sample_GLOVE', column, 'random'+'-'+str(SLURM_JOB_ID)+'-'+str(SLURM_ARRAY_TASK_ID)+'.csv'), 'saved')

print('time taken:', str(time.time() - start_time), 'seconds')


# break

************ job_offer ************
     id  \
0  3256   
1  6090   
2  5811   
3  1085   
4  2383   

                                                                                                                                       text  \
0  Just fired up my GPS from 1999. New batteries and in 30 seconds it found a position indoors! http://t.co/gWZXkx2L2w                        
1  @MysteryColorist its a fuckin CRIME!! They just charge batts turn the thing on &amp; go. YET STILL GET HIRED FOR SLOPPY WORK!              
2  WAIT WHAT ? WHO QUITS THEIR JOB BEFORE THEY GET AN INTERVIEW WITH THE NEXT COMPANY?....                                                    
3  Searching For a LinkedIn Marketing Superstar! - oDesk: I am seeking an experience LinkedIn ... http://t.co/tK73BjLJ6H #sales #mktg #jobs   
4  Who is hiring???                                                                                                                           

   class  
0  0      
1  0      
2  0 

In [4]:
df_random

,neg_model,pos_model
tweet_id,,
554664286443028480,0.807784,0.192216
554666350086737920,0.718504,0.281496
554666446463438848,0.999905,0.000095
554667234845806592,0.014462,0.985538
554668283765354498,0.878246,0.121754
...,...,...
554797904561332225,0.253242,0.746758
554797921049120768,0.203928,0.796072
554798437204779008,0.000521,0.999479


In [12]:
pd.DataFrame(data=predictions_random, columns = ['neg', 'pos'])

,neg,pos
0,0.215886,0.784114
1,0.389961,0.610039
2,0.005759,0.994241
3,0.998955,0.001045
4,0.141725,0.858275
...,...,...
95,0.399320,0.600680
96,0.541336,0.458664
97,0.149652,0.850348
98,0.991346,0.008654
